In [1]:
from tensorflow.keras.applications import Xception, DenseNet201, MobileNetV2, DenseNet169
from tensorflow.keras.layers import Softmax, ReLU, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling1D, Flatten, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, Input
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow import convert_to_tensor
from sklearn.model_selection import train_test_split
from glob import glob
from PIL import Image
from os import path
from sklearn.utils import shuffle
import numpy as np
from tensorflow import one_hot
import tensorflow as tf

2022-05-23 18:43:07.419988: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 18:43:07.420008: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
image_shape = (224, 224, 3)
train_depth = 10
batch_size = 32
epochs = 10
steps_per_epoch=20
save_path = './'

In [3]:
def get_pictures(data_path):
    data = []
    labels = []
    species = sorted(glob(path.join(data_path, "*")))
    for label_id, sname in enumerate(species):
        image_names = sorted(glob(path.join(sname, "*")))
        for iname in image_names:
            data.append(np.array(Image.open(iname).convert("RGB")))
            labels.append(label_id)
    return np.asarray(data, dtype='float32'), np.asarray(labels)

In [4]:
base_model = Xception(input_tensor=Input(shape=image_shape), include_top=False)
base_model.trainable = False

cnt = train_depth
while cnt > 0:
    base_model.layers[-cnt].trainable = True
    cnt -= 1
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(400))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.3))
model.add(Dense(12, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

2022-05-23 18:43:10.369401: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-23 18:43:10.369435: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vlad-desktop): /proc/driver/nvidia/version does not exist
2022-05-23 18:43:10.369725: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
X_train, y_train = get_pictures("data/train")
X_test, y_test = get_pictures("data/test")

X_train = np.array(X_train)
X_test = np.array(X_test)

# X_train = tf.convert_to_tensor(X_train)
# X_test = tf.convert_to_tensor(X_test)
n_classes = np.unique(y_train).shape[0]
y_train = one_hot(y_train, n_classes)
y_test = one_hot(y_test, n_classes)

X_train = X_train / 255.0
X_test = X_test / 255.0
# X_train = (X_train - X_train.mean(axis=(1, 2, 3))) / X_train.std(axis=(1, 2, 3))
# X_test = (X_test - X_test.mean(axis=(1, 2, 3))) / X_test.std(axis=(1, 2, 3))

In [6]:
y_train

<tf.Tensor: shape=(1536, 12), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [7]:
X_train[0].shape

(224, 224, 3)

In [8]:
y_test.shape

TensorShape([300, 12])

In [9]:
X_train.shape

(1536, 224, 224, 3)

In [10]:
X_test.shape

(300, 224, 224, 3)

In [11]:
data_gen = ImageDataGenerator(zoom_range=0.15, horizontal_flip=True, rotation_range=15, 
                              height_shift_range=0.3, width_shift_range=0.2)
train_generator = data_gen.flow(X_train, y_train, batch_size=batch_size, shuffle=True)
test_generator = data_gen.flow(X_test, y_test, batch_size=batch_size)

In [12]:
model.fit(train_generator, epochs=epochs, validation_data=test_generator, verbose=1)

Epoch 1/10
48/48 [==============================] - 113s 2s/step - loss: 1.5223 - accuracy: 0.5469 - val_loss: 1.2256 - val_accuracy: 0.8367
Epoch 2/10
48/48 [==============================] - 110s 2s/step - loss: 0.5103 - accuracy: 0.8743 - val_loss: 0.8286 - val_accuracy: 0.8633
Epoch 3/10
48/48 [==============================] - 118s 2s/step - loss: 0.3306 - accuracy: 0.9219 - val_loss: 0.4802 - val_accuracy: 0.9500
Epoch 4/10
48/48 [==============================] - 128s 3s/step - loss: 0.2773 - accuracy: 0.9395 - val_loss: 0.3401 - val_accuracy: 0.9667
Epoch 5/10
48/48 [==============================] - 139s 3s/step - loss: 0.2036 - accuracy: 0.9577 - val_loss: 0.2475 - val_accuracy: 0.9500
Epoch 6/10
48/48 [==============================] - 128s 3s/step - loss: 0.1902 - accuracy: 0.9570 - val_loss: 0.1537 - val_accuracy: 0.9633
Epoch 7/10
48/48 [==============================] - 124s 3s/step - loss: 0.1757 - accuracy: 0.9564 - val_loss: 0.1571 - val_accuracy: 0.9533
Epoch 8/10
48

2022-05-23 19:05:04.852559: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./assets


In [13]:
model.save("./model10epoches.hdf5")

In [ ]:
def test_model(model, test_data_path, n_classes):
    X_test, y_test = get_pictures(test_data_path)
    X_test = X_test / 255.0
    y_test = one_hot(y_test, n_classes)
    model.evaluate(X_test, y_test)

test_model(model=model, test_data_path="data/test", n_classes=n_classes)

In [37]:
def predict_stolen_labels(model, test_data_path, n_classes):
    X_test, y_test = get_pictures(test_data_path)
    X_test = X_test / 255.0
    y_test = one_hot(y_test, n_classes)
    return model.predict(X_test)
predict_stolen_labels = predict_stolen_labels(model=model, test_data_path="data/stolen_train", n_classes=n_classes)

2022-05-23 20:25:52.788030: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1104875520 exceeds 10% of free system memory.


In [38]:
y_test_stolen = np.argmax(predict_stolen_labels, axis=1)

In [39]:
y_test_stolen = one_hot(y_test_stolen, n_classes)

In [40]:
base_model2 = Xception(input_tensor=Input(shape=image_shape), include_top=False)
base_model2.trainable = False

cnt = train_depth
while cnt > 0:
    base_model2.layers[-cnt].trainable = True
    cnt -= 1
model2 = Sequential()
model2.add(base_model2)
model2.add(GlobalAveragePooling2D())
model2.add(Dense(400))
model2.add(BatchNormalization())
model2.add(ReLU())
model2.add(Dropout(0.3))
model2.add(Dense(12, activation='softmax'))

model2.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [41]:
X_train, y_train = get_pictures("data/stolen_train")

In [42]:
X_train = X_train / 255.0

In [43]:
data_gen = ImageDataGenerator(zoom_range=0.15, horizontal_flip=True, rotation_range=15, 
                              height_shift_range=0.3, width_shift_range=0.2)
train_generator = data_gen.flow(X_train, y_test_stolen, batch_size=batch_size, shuffle=True)
test_generator = data_gen.flow(X_test, y_test, batch_size=batch_size)

ValueError: `x` (images tensor) and `y` (labels) should have the same length. Found: x.shape = (1964, 224, 224, 3), y.shape = (1835, 12)

In [35]:
model2.fit(train_generator, epochs=epochs, validation_data=test_generator, verbose=1)

Epoch 1/10
62/62 [==============================] - 146s 2s/step - loss: 1.7386 - accuracy: 0.4842 - val_loss: 1.6957 - val_accuracy: 0.5600
Epoch 2/10
62/62 [==============================] - 166s 3s/step - loss: 0.9515 - accuracy: 0.7138 - val_loss: 1.4595 - val_accuracy: 0.5333
Epoch 3/10
62/62 [==============================] - 145s 2s/step - loss: 0.8312 - accuracy: 0.7429 - val_loss: 1.2076 - val_accuracy: 0.6333
Epoch 4/10
62/62 [==============================] - 139s 2s/step - loss: 0.7896 - accuracy: 0.7510 - val_loss: 1.0748 - val_accuracy: 0.6900
Epoch 5/10
62/62 [==============================] - 149s 2s/step - loss: 0.7653 - accuracy: 0.7469 - val_loss: 1.0642 - val_accuracy: 0.6933
Epoch 6/10
62/62 [==============================] - 175s 3s/step - loss: 0.6952 - accuracy: 0.7704 - val_loss: 1.1230 - val_accuracy: 0.6833
Epoch 7/10
62/62 [==============================] - 161s 3s/step - loss: 0.7011 - accuracy: 0.7576 - val_loss: 0.9125 - val_accuracy: 0.7600
Epoch 8/10
62

In [36]:
model2.fit(train_generator, epochs=epochs, validation_data=test_generator, verbose=1)

Epoch 1/10
62/62 [==============================] - 167s 3s/step - loss: 0.5782 - accuracy: 0.7892 - val_loss: 1.0214 - val_accuracy: 0.7367
Epoch 2/10
62/62 [==============================] - 164s 3s/step - loss: 0.5555 - accuracy: 0.8070 - val_loss: 0.9743 - val_accuracy: 0.7600
Epoch 3/10
62/62 [==============================] - 184s 3s/step - loss: 0.5633 - accuracy: 0.8004 - val_loss: 0.9469 - val_accuracy: 0.7767
Epoch 4/10
62/62 [==============================] - 198s 3s/step - loss: 0.5400 - accuracy: 0.8218 - val_loss: 0.9311 - val_accuracy: 0.7833
Epoch 5/10
62/62 [==============================] - 182s 3s/step - loss: 0.5386 - accuracy: 0.8060 - val_loss: 1.0384 - val_accuracy: 0.7667
Epoch 6/10
62/62 [==============================] - 159s 3s/step - loss: 0.5593 - accuracy: 0.8101 - val_loss: 0.9735 - val_accuracy: 0.7633
Epoch 7/10
62/62 [==============================] - 157s 3s/step - loss: 0.5047 - accuracy: 0.8243 - val_loss: 1.1136 - val_accuracy: 0.7433
Epoch 8/10
62

In [29]:
X_train.shape

(1964, 224, 224, 3)

In [30]:
y_test_stolen.shape

TensorShape([1964, 12])

In [ ]:
model2.fit(train_generator, epochs=epochs, validation_data=test_generator, verbose=1)